### Use ``Open Route Service API`` to retrieve distance matrix (Single Call version)
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 
#### The following processes 2022 separately since no place data is available for 2022, by assumption we use the most recent 2021 instead

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import openrouteservice as ors
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/result_place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(75786, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2021]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude']).reset_index(drop=True)
print(place_df.shape)
place_df.head(10)

(9283, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,WEST MELBOURNE MARKET,-37.81040,144.92237,2021,market,206041127
1,LILYDALE MARKET,-37.76040,145.34931,2021,market,211051278
2,PRESTON MARKET,-37.73790,145.00264,2021,market,209021428
3,PRAHRAN MARKET,-37.84615,144.99235,2021,market,206061516
4,VICTORIA MARKET,-37.80651,144.95653,2021,market,206041504
5,SOUTH MELBOURNE MARKET,-37.83263,144.95681,2021,market,206051512
6,CROYDON MARKET,-37.79374,145.27820,2021,market,211031451
7,SWAN HILL HOSPITAL,-35.34040,143.55626,2021,hospital,215031404
8,CLUNES HOSPITAL,-37.30179,143.78265,2021,hospital,201021010
9,MALDON HOSPITAL,-36.99270,144.06400,2021,hospital,202021028


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2_classified/2022_property_with_SA2_classified.csv')[PROPERTY_COLS]
# 2022 example
print(property_df.shape)
property_df.head(10)

(73107, 12)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,postcode,year,month,residence_type,SA2_CODE
0,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070
1,"17 PARK STREET, CHILTERN",-36.148025,146.608721,3.0,2.0,0,350,3683,2022,8,House,204031070
2,"3/1 OXFORD STREET, CHILTERN",-36.150694,146.606488,2.0,1.0,1,300,3683,2022,4,Apartment,204031070
3,"2/20 NICKLESS STREET, CHILTERN",-36.154038,146.602977,2.0,1.0,2,350,3683,2022,4,Apartment,204031070
4,"1/3 BURKE STREET, CHILTERN",-36.153004,146.607465,2.0,1.0,0,240,3683,2022,2,Apartment,204031070
5,"590 KENNEDYS LANE, CHINTIN",-37.414852,144.825852,4.0,2.0,7,850,3756,2022,8,House,210021235
6,"4 SHERWOOD ROAD, CHIRNSIDE PARK",-37.747341,145.332546,4.0,2.0,2,680,3116,2022,8,House,211051275
7,"4 SHERWOOD ROAD, CHIRNSIDE PARK",-37.747341,145.332546,4.0,2.0,2,680,3116,2022,9,House,211051275
8,"4 GLENDALE DRIVE, CHIRNSIDE PARK",-37.751383,145.312411,4.0,2.0,2,480,3116,2022,8,House,211051275
9,"4 GLENDALE DRIVE, CHIRNSIDE PARK",-37.751383,145.312411,4.0,2.0,2,480,3116,2022,9,House,211051275


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(f"shape = {merged_df.shape}")
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

shape = (1351949, 17)
Distinct sa2 codes = 502


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN POLICE STATION,-36.14815,146.61057,police
1,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN PRIMARY SCHOOL,-36.14707,146.60660,primary
2,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,MIDDLE INDIGO PRIMARY SCHOOL,-36.15601,146.72484,primary
3,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,BARNAWARTHA PRIMARY SCHOOL,-36.10131,146.67589,primary
4,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,ST JOSEPHS SCHOOL,-36.14598,146.60707,primary
5,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN,-36.15566,146.61138,station
6,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,BARNAWARTHA,-36.10280,146.67959,station
7,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN RAILWAY STATION,-36.15540,146.61181,station
8,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN VALLEY RECREATION RESERVE,-36.14293,146.54548,park
9,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,3683,2022,9,House,204031070,CHILTERN PARK,-36.12429,146.55681,park


### Create the directory to store merged distance data

In [6]:
if not os.path.exists('../../data/featured'):
    os.makedirs('../../data/featured')

#### Specify the ``client`` and ``year`` here, as well as adding a mode of testing/saving

In [7]:
client = ors.Client('5b3ce3597851110001cf6248e33390389e0b4e0292fb3dcd2dec286b') # Phikhocaz's api key 500

In [8]:
subset = merged_df[merged_df['SA2_CODE'] == 203021039].reset_index(drop=True)
subset

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.181740,144.367010,1.0,1.0,1,250,3219,2022,8,Apartment,203021039,GEELONG HOSPITAL,-38.15235,144.36376,hospital
1,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.181740,144.367010,1.0,1.0,1,250,3219,2022,8,Apartment,203021039,GEELONG POLICE STATION,-38.14536,144.35692,police
2,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.181740,144.367010,1.0,1.0,1,250,3219,2022,8,Apartment,203021039,GEELONG HIGH SCHOOL,-38.15303,144.37373,secondary
3,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.181740,144.367010,1.0,1.0,1,250,3219,2022,8,Apartment,203021039,MATTHEW FLINDERS GIRLS SECONDARY COLLEGE,-38.14959,144.35321,secondary
4,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.181740,144.367010,1.0,1.0,1,250,3219,2022,8,Apartment,203021039,MATTHEWS FLINDERS GIRLS HIGH SCHOOL,-38.14959,144.35321,secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11275,"12 ROMLEY AVENUE, ARMSTRONG CREEK",-38.168527,144.377881,4.0,2.0,2,400,3217,2022,5,House,203021039,CRANE COURT PLAYGROUND,-38.16308,144.35840,park
11276,"12 ROMLEY AVENUE, ARMSTRONG CREEK",-38.168527,144.377881,4.0,2.0,2,400,3217,2022,5,House,203021039,GEORGE TRICKEY PLAYGROUND,-38.16089,144.36120,park
11277,"12 ROMLEY AVENUE, ARMSTRONG CREEK",-38.168527,144.377881,4.0,2.0,2,400,3217,2022,5,House,203021039,ORCHARD PLAYGROUND,-38.17893,144.37965,park
11278,"12 ROMLEY AVENUE, ARMSTRONG CREEK",-38.168527,144.377881,4.0,2.0,2,400,3217,2022,5,House,203021039,STRADLING RESERVE,-38.13648,144.35366,park


In [9]:
added_distance_merged_df = add_distance_time(subset, 2022, client, 'testing') # specify the year here, client id, and mode of running (testing or saving)
#added_distance_merged_df = add_distance_time(subset, 2013)

Executing SA2 Code 203021039
Subset size = 11280, Places count = 40, Property count = 282
To Place Distance Grand List, Slice Branch, length = 2800
To Place Distance Grand List, Slice Branch, length = 5600
To Place Distance Grand List, Slice Branch, length = 8400
To Place Distance Grand List, Slice Branch, length = 11280


In [10]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,postcode,year,month,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.36376,hospital,3901.93,3.90193,390.24,6.504000,83535.83,83.53583,4372.13,72.868833
1,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.35692,police,5379.61,5.37961,566.21,9.436833,83535.83,83.53583,4372.13,72.868833
2,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.37373,secondary,3464.87,3.46487,355.98,5.933000,83535.83,83.53583,4372.13,72.868833
3,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.35321,secondary,4819.79,4.81979,462.17,7.702833,83535.83,83.53583,4372.13,72.868833
4,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.35321,secondary,4819.79,4.81979,462.17,7.702833,83535.83,83.53583,4372.13,72.868833
5,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.38237,secondary,2379.68,2.37968,222.60,3.710000,83535.83,83.53583,4372.13,72.868833
6,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.38237,secondary,2379.68,2.37968,222.60,3.710000,83535.83,83.53583,4372.13,72.868833
7,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.36742,secondary,4575.62,4.57562,499.18,8.319667,83535.83,83.53583,4372.13,72.868833
8,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.38277,primary,2064.54,2.06454,188.75,3.145833,83535.83,83.53583,4372.13,72.868833
9,"UNIT 2, 61 TUCKER STREET, BREAKWATER",-38.18174,144.36701,1.0,1.0,1,250,3219,2022,8,...,144.35857,primary,3491.26,3.49126,292.58,4.876333,83535.83,83.53583,4372.13,72.868833
